In [ ]:
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy import optimize
import pandas as pd
import seaborn as sns
from pandas.core.frame import DataFrame
import seaborn as sns
import math 
from scipy.optimize import curve_fit
import xarray as xr
import pymannkendall as mk
from tqdm import tqdm
filePath = 'C:/Users/***/Desktop/500_100/789'
filename = os.listdir(filePath)
precip_all = np.zeros((15388,59*92))
for i in range(15388):
    path='C:/Users/***/Desktop/500_100/789/{}'.format(filename[i])
    precip = pd.read_csv(path)
    precip_all[i] = precip['prep']
precip_max = np.nanmax(precip_all,axis=0)
precip_max = np.where(np.isnan(precip_max), 0, precip_max)
precip_2d = precip_max.reshape((59,92))
fig = plt.figure(figsize=(12,8))
ax = fig.add_axes([0.1,0.1, 0.9, 0.8])
c1 = ax.contourf(np.arange(0,92,1), np.arange(1961,2020,1), precip_2d, zorder=0, levels = [0,10,25,50,100,150,205,275],cmap = 'YlGnBu_r',extend='both')#list(range(10,110,10)) viridis
ax.set_title('Daily Max Preciptation in China(JUN-AUG)', fontsize=22, pad = 30, loc='center', weight='bold')
ax.set_xticks([0,14,31,46,61,76,91])
ax.set_xticklabels(['Jun1','Jun15','Jul1','Jul15','Aug1','Aug15','Aug31'], fontsize = 12)
ax.set_yticks([1961,1970,1980,1990,2000,2010,2019])
ax.set_yticklabels(['1961','1970','1980','1990','2000','2010','2019'], fontsize = 12)
ax.set_ylabel('year',fontsize=18)
fig.colorbar(c1)
c2 = ax.contour(np.arange(0,92,1),np.arange(1961,2020,1), precip_2d, zorder=0,levels =[150],colors=['r'],linestyles=['--']) #'aqua'
plt.savefig('Daily Max Preciptation in China(JUN-AUG).jpg',dpi = 1000)

In [ ]:
date = precip['time']
date_df = DataFrame({"date":date.values,'precip':list(precip_max)})
date_df_3 = date_df[date_df.precip >= 65]
n = date_df_3.values.shape[0]
date_df_3.index = range(n)
n_1 = date_df_3['date'].values
frequency = []
for i in range(len(n_1)):
    frequency.append(int(str(n_1[i])[0:4]))
dict_1 = {}
for key in frequency:
    dict_1[key] = dict_1.get(key,0)+1
years = list(dict_1.keys())
frequency = list(dict_1.values())
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_axes([0.1, 0.1, 0.8, 0.8])
plt.xticks(years)
ax1.set_xticks([1961,1965,1970,1975,1980,1985,1990,1995,2000,2005,2010,2015,2019])
ax1.set_xticklabels(['1961','1965','1970','1975','1980','1985','1990','1995','2000','2005','2010','2015','2019'], fontsize = 14)
ax1.set_yticks([30,35,40,45,50,55,60,65,70,75])
ax1.set_yticklabels(['30','35','40','45','50','55','60','65','70','75'], fontsize = 14)
ax1.set_ylabel('Frequencies',fontsize=16)
ax1.tick_params(axis='y',colors='navy',labelsize=12)
ax1.set_title('Yearly Frequencies of extreme preciptation days',fontsize=22, pad = 20, weight='bold')
ax1.plot(years, frequency,'b--')
sns.set(style='darkgrid')
sns.regplot(years, frequency, color='g')
plt.savefig('Yearly Frequencies of extreme preciptation days.tiff',dpi = 1000)

In [ ]:
date = precip['time']
date_df = DataFrame({"date":date.values,'precip':list(precip_max)})
date_df_2 = date_df[date_df.precip >= 100]
date_df_2['date'] = pd.to_datetime(date_df_2['date'])
date_df_2['dt'] = date_df_2['date'].shift(-1)-date_df_2['date']
date_df_2['dt'] = date_df_2['dt'].dt.days
date_df_2['dt'][date_df_2['dt']!=1] = 0
date_df_2['dt'] = date_df_2['dt'].astype('int')
duration = np.zeros_like(date_df_2['dt'])
date_df_2['duration'] = duration
count=0
for i, dt_value in enumerate(date_df_2['dt']):
    if dt_value == 0:
        duration[i]=count+1
        count=0
    else:
        count+=1
date_df_2['duration'] = duration
delta = []
for i in range(date_df_2['date'].shape[0]):
    delta.append(np.timedelta64(date_df_2['duration'].values[i]-1,'D'))
date_df_2['dt'] = delta
date_df_2['start_date'] = date_df_2['date'] - date_df_2['dt']
n = date_df_2.values.shape[0]
date_df_2.index = range(n)
intensity = np.zeros((n))
for i in range(n):
    if date_df_2.at[i,'duration'] == 0 or date_df_2.at[i,'duration'] == 1:
        intensity[i] = date_df_2.at[i,'precip']
    else:
        intensity[i] = sum(date_df_2.loc[i-date_df_2.at[i,'duration']+1:i,'precip'])
date_df_2['intensity'] = intensity
date_df_2 = date_df_2[date_df_2.duration>1]
n = date_df_2.values.shape[0]
date_df_2.index = range(n)
intensity = date_df_2['intensity'].values
duration = date_df_2['duration'].values
fig = plt.figure(figsize=(10,8))
ax1 = fig.add_axes([0.1, 0.1, 1.6, 1])
ax1.bar(np.arange(n),duration,color='lightskyblue',edgecolor='skyblue',width = 0.5)
labels = list(np.arange(n))
plt.xticks(range(0,n,20), fontsize=15)
plt.xlabel('Cases (times)',fontsize=18)
ax1.set_ylabel('Duration/days (bar)',fontsize=18)
ax1.tick_params(axis='y',colors='lightskyblue')
plt.tick_params(labelsize=15) 
ax2 = ax1.twinx()
ax2.plot(np.arange(n), intensity, color='orangered', marker='*', linestyle='None') #linestyle='None'
ax2.set_ylabel('Preciptation/mm (line)',fontsize=18)
ax2.tick_params(axis='y',colors='orangered')
ax2.set_title('Extreme Preciptation intensity & duration',fontsize=22, pad = 20, weight='bold')
plt.tick_params(labelsize=14) 
plt.savefig('Extreme Preciptation intensity & duration.tiff', dpi = 1000, bbox_inches = 'tight')